<img src="http://dkrib.web.elte.hu/compare/COMPARE_LOGO.jpg" width=400 img>

<div class="jumbotron">

<center> <h2> Demo notebook  </h2> </center>

<center> <h1> Query all the geographic location from EBI Influenza A samples and draw them on a map  </h1> </center>

</div>

---
#About the Jupyter-notebook


####The programming languages
- This notebook is written in python, but you can use the exact same jupyter framework in many different languages (R,Ruby,Julia,Haskell etc..). Please explore the jupyter project webpage for more information about support for programming languages

<center>
<a type="button" class="btn btn-lg btn-warning " href="https://jupyter.org/" >Link to Jupyter project</a>
</center>



####This is a markdown cell
- You can write easy markdown headers and notes like this
- Or you can write html ike the jumbotron above. In the notebook, you can use the Bootstrap framework to have nice buttons etc. like the one below.

<center>
<a type="button" class="btn btn-lg btn-success " href="http://getbootstrap.com/" >Link to Bootstrap</a>
</center>

- You can also write equations, which will be rendered by [MathJax](http://www.mathjax.org/)

$$E = mc^2$$

---
#The purpose of this small demo is to explore Influenza A geolocations in the [ENA](http://www.ebi.ac.uk/ena)

###First we should figure, how many flu samples are there?

- ENA has an advanced search option where we can discover data with some filtering. It has a graphical interface but, it also support programatic acces through url based queries.
- [Advanced search graphical interface](http://www.ebi.ac.uk/ena/data/warehouse/search)
- [Advanced search tutorial ](http://www.ebi.ac.uk/ena/support/advanced-search-tutorial)

I will build the url from the logical blocks:
- the thing below is called a code cell, If you push Shift+Enter, or click the triangle at the menubar, the code will be executed

In [1]:
url_base='http://www.ebi.ac.uk/ena/data/warehouse/search?' #base for advanced search
url_query='\"tax_tree(11320)\"' #influenza A taxon and all subordinates (tree)
url_result='&result=sample' # looking for samples, they have location
url_count='&resultcount' # count the results

url=url_base+url_query+url_result+url_count #concatenate

print 'The url is:',url #print

The url is: http://www.ebi.ac.uk/ena/data/warehouse/search?"tax_tree(11320)"&result=sample&resultcount


Query the url, read the result back as a string
- Actually you can also click on it, and you will be presented with the results int the browser

In [2]:
import urllib #python modules for url-s
print urllib.urlopen(url).read()

Number of results: 1,097,437
Time taken: 0 seconds


###Now i will download all the geolocation information associated with the samples

Build url again

In [3]:
url_base='http://www.ebi.ac.uk/ena/data/warehouse/search?'
url_query='\"tax_tree(11320)\"'
url_result='&result=sample'
url_display='&display=report' #report is the tab separated output
url_fields='&fields=accession,location' #get accesion and location
url_limits='&offset=1&length=1095067' #get all the results

url=url_base+url_query+url_result+url_display+url_fields+url_limits

The result is a tab separated table
- I will Download the table to a string

In [4]:
ena_flu_loco_page = urllib.urlopen(url).read()

Load the table into a pandas DataFrame
- [Pandas](http://pandas.pydata.org/) is a very useful library for data analysis in python
- The DataFrame object is similar to R dataframes

In [5]:
import pandas as pd #pandas
from StringIO import StringIO #for reading string into pandas
ena_flu_loco_table = pd.read_csv(StringIO(ena_flu_loco_page),sep='\t')

Peek into the table
- Unfortunately most of the values are missing (NaNs)

In [6]:
ena_flu_loco_table.head()

,accession,location
0,33124,NaN
1,9544,NaN
2,9545,NaN
3,SAMD00000344,NaN
4,SAMD00000345,NaN


###See how many  geolocation data is there?

In [7]:
print "The number of sample with geolocations is: ",
print len(ena_flu_loco_table[ pd.isnull(ena_flu_loco_table['location']) == False ])

The number of sample with geolocations is:  125524


Get rid of samples with no geolocation

In [8]:
ena_flu_loco_table=ena_flu_loco_table[ pd.isnull(ena_flu_loco_table['location']) == False ]

###Some location are malformed

In [9]:
err= ena_flu_loco_table[ [ len(x.split(' '))!=4 for x in ena_flu_loco_table['location'] ]]
err.head()

,accession,location
196957,SAMEA2384944,18.49041 E
196981,SAMEA2384968,18.49041 E
202131,SAMEA2392363,23.1 E
234589,SAMEA2547925,28.23 E
234590,SAMEA2547926,28.23 E


Delete these

In [10]:
ena_flu_loco_table = ena_flu_loco_table[ [ 
        len(x.split(' '))==4 for x in ena_flu_loco_table['location'] ]]

###Parse the longitudes, longitudes
- The data is in a different format than the map will need read, so I need to convert is. (N,E,S,W) instead of negative values

In [11]:
def parse_lat(string_loc):
    loc_list=string_loc.split(' ')
    if (loc_list[1] =='N'):
        return float(loc_list[0])
    elif (loc_list[1] =='S'):
        return -float(loc_list[0])
    
def parse_lon(string_loc):
    loc_list=string_loc.split(' ')
    if (loc_list[3] =='E'):
        return float(loc_list[2])
    elif (loc_list[3] =='W'):
        return -float(loc_list[2])
    
ena_flu_loco_table['lat']=map(parse_lat,ena_flu_loco_table['location'])
ena_flu_loco_table['lon']=map(parse_lon,ena_flu_loco_table['location'])

ena_flu_loco_table=ena_flu_loco_table[['lat','lon','accession']]

In [12]:
ena_flu_loco_table.head()

,lat,lon,accession
2856,44.337,143.38083,SAMD00003553
2857,44.337,143.38083,SAMD00003554
2858,44.337,143.38083,SAMD00003555
2859,44.337,143.38083,SAMD00003556
2860,44.337,143.38083,SAMD00003557


###See how many unique locations we have

In [13]:
print 'Number of unique locations:',
print len(ena_flu_loco_table.groupby(['lat','lon']).size().reset_index())

Number of unique locations: 16146


###Generate a popup string for each unique location
- This will be shown on the map, when you click on the point with the mouse

Contents:
- Number of cases
- list of accession numbers, truncated if too long

I am using the sql-like groupby statement for group the samples

In [14]:
#the function used for grouping
def form_acc(x):
    if (x['accession'].size < 5):
        return pd.Series(
            dict({'count' : x['accession'].size, 'acc_list' : ' '.join(x['accession']),
                }))
    else:
        return pd.Series(
            dict({'count' : x['accession'].size, 'acc_list' : ' '.join(list(
                        x['accession'])[:2]) + ' ... ' + ' '.join(list(
                        x['accession'])[-2:])}))

#group-by
uniq_locs_w_acc=ena_flu_loco_table.groupby(['lat','lon']).apply(form_acc).reset_index()

<div class="jumbotron">

<center> <h1> NOTE: UNITL NOW, THE NOTEBOOK IS EXACTLY THE SAME AS THE POINT DRAWING NOTEBOOK  </h1> </center>

</div>

---
#Create map overlay

Download the geojson for US-states
- This data is from the github of the author of Folium : https://github.com/wrobstory

In [15]:
#Download Us states geojson
import urllib
page=urllib.urlopen('https://gist.github.com/wrobstory/5609856/raw/3d6a5d15fc23d286f6d4404ce9b705a91fd04676/us-states.json').read()
f=open('us-states.json','w')
f.write(page)
f.close()

Read the state polygons from the GeoJson file

In [16]:
import json
import shapely.geometry

#read the json
f=file('us-states.json', 'r')
js = json.loads(f.read())
f.close()

#get states
dict_list=[]
for f in js['features']:
    dict_list.append(dict({'state_id': f['id'],
                           'shape': shapely.geometry.shape(f['geometry']) }))

Assign each point to a state polygon
- This is very naive 
- <span class="label label-warning">Note</span>  This is very slow, It would be unacceptably slow with and order of magnite more polygons (all countries, us counties, etc)

In [17]:
#dataframe the states
states_df=pd.DataFrame(dict_list)

#find the polygon for each point naively
states_df['no_cases']=0
for i in xrange(len(uniq_locs_w_acc)):
    
    temp_loc=(uniq_locs_w_acc.iloc[i]['lon'],uniq_locs_w_acc.iloc[i]['lat'] )
    temp_point=shapely.geometry.Point(temp_loc)
    
    for j in xrange(len(states_df)):
        try:
            shapely.geometry.Polygon(states_df.loc[j,'shape']) #throws excetion if not polygon     
            if (states_df.loc[j,'shape'].contains(temp_point)):
                states_df.loc[j,'no_cases']+=uniq_locs_w_acc.iloc[i]['count'] 
                break
        except:
            pass

Write the layer data in json for Folium
- this will be read by the javascript map drawing script

In [23]:
#dump it in json
dump_df=states_df[['state_id','no_cases']]
dump_f=open('map_data.json','w')
dump_f.write(json.dumps([dict(dump_df.to_dict(orient='split')['data'])])) #the good format for Folium
dump_f.close()


---
#Draw the map, with the overlay

I will use the [Folium](http://folium.readthedocs.org/en/latest/) library which is python wrapper for the [Leaflet](http://leafletjs.com/) javasript library for map based visualizations


First define the inline map drawing function

- solution from here : https://ocefpaf.github.io/python4oceanographers/blog/2014/05/05/folium/
- Puts the map in an iframe in the output cell

In [24]:
from IPython.core.display import HTML
import folium

def inline_map(m, width=1000, height=600):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed

Initialize the map

In [28]:
data_map=folium.Map(location=(38,-95),tiles='Mapbox Bright', zoom_start=4)

Add GeoJson layer, with data binded
- This should work without writing out the data to the 'temp_data.json' data file, but it does not work for me
    - see  examples: https://github.com/python-visualization/folium

In [29]:
data_map.geo_json(geo_path='us-states.json',
                  data=states_df,
                  data_out='map_data.json',
                  columns=['state_id','no_cases'],
                  key_on='feature.id',
                  fill_color='BuGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Number of Influenza A samples in ENA from each state',
                  reset=True)

Show the map

In [30]:
inline_map(data_map)

---
#Some notes about this notebook:

Memory footprint: 
- python: 220 MiB
- chrome: 400 MiB

Runtime:
- very slow: point -> states takes 30 minites


Map:
- Pandas binding deos not work flawlessly, i have to wrtie out data in json by hand
- More detaileg GeoJson breaks the map (us counties)
- Very limited colormap
    - max 6 values ?
    - always starts from 0?